1. '근로'에 대한 판례내용에서 가장 자주 나타나는 소송 주제 찾기
2. 다음의 열에 있는 값들과 상관관계 알고 싶어 
    -열 이름 : 법원명, 사건종류명, 재판당사자
3. '근로'에 대한 판례에서 가장 자주 쓰이는 참조판례, 참조조문
   -열 이름 : 참조조문, 참조판례

1. '근로'에 대한 판례내용에서 가장 자주 나타나는 소송 주제 찾기

In [ ]:
# 1. '근로'에 대한 판례내용에서 가장 자주 나타나는 소송 주제 찾기
def find_frequent_cases(data, keyword='근로', num_terms=10):
    filtered_data = data[data['판례내용'].str.contains(keyword)]
    all_terms = ' '.join(filtered_data['processed'])
    term_counts = Counter(all_terms.split())
    most_common_terms = term_counts.most_common(num_terms)
    
    terms, counts = zip(*most_common_terms)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=list(terms))
    plt.title(f'Top {keyword} Related Legal Terms Frequency')
    plt.xlabel('Frequency')
    plt.ylabel('Terms')
    plt.show()

find_frequent_cases(data)

2. 법원명, 사건종류명, 재판당사자와 '근로'에 대한 판례 내용의 상관관계 분석

In [ ]:
# 2. 법원명, 사건종류명, 재판당사자와 상관관계 분석
def analyze_correlation(data, keyword='근로'):
    filtered_data = data[data['판례내용'].str.contains(keyword)]
    
    # 법원명, 사건종류명, 재판당사자와 '근로' 관련 판례 내용의 빈도 수 계산
    correlation_data = filtered_data[['법원명', '사건종류명', '재판당사자']]
    
    # 각 열의 고유값 수 카운트
    correlation_counts = correlation_data.apply(lambda x: x.value_counts()).fillna(0)
    
    # 상관관계 시각화
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_counts, annot=True, cmap='coolwarm')
    plt.title(f'Correlation Analysis of Court, Case Type, and Parties related to {keyword}')
    plt.show()

analyze_correlation(data)


3. '근로'에 대한 판례에서 가장 자주 쓰이는 참조판례, 참조조문

In [ ]:
# 3. '근로'에 대한 판례에서 가장 자주 쓰이는 참조판례, 참조조문
def find_frequent_references(data, keyword='근로', num_terms=10):
    filtered_data = data[data['판례내용'].str.contains(keyword)]
    
    # 참조판례와 참조조문의 빈도 계산
    reference_cases = filtered_data['참조판례'].str.split(',').explode().str.strip().dropna()
    reference_articles = filtered_data['참조조문'].str.split(',').explode().str.strip().dropna()
    
    case_counts = Counter(reference_cases)
    article_counts = Counter(reference_articles)
    
    # 참조판례 시각화
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    most_common_cases = case_counts.most_common(num_terms)
    cases, case_frequencies = zip(*most_common_cases)
    sns.barplot(x=list(case_frequencies), y=list(cases))
    plt.title('Top Referenced Cases')
    plt.xlabel('Frequency')
    plt.ylabel('Cases')

    # 참조조문 시각화
    plt.subplot(1, 2, 2)
    most_common_articles = article_counts.most_common(num_terms)
    articles, article_frequencies = zip(*most_common_articles)
    sns.barplot(x=list(article_frequencies), y=list(articles))
    plt.title('Top Referenced Articles')
    plt.xlabel('Frequency')
    plt.ylabel('Articles')

    plt.tight_layout()
    plt.show()

find_frequent_references(data)


재판당사자 열에서 단어간의 앞뒤 상관관계를 분석

데이터 준비: 재판당사자 열에서 데이터를 로드합니다.
단어 쌍 생성: 각 재판당사자 항목에서 단어 간의 쌍을 생성합니다.
빈도 분석: 생성된 단어 쌍의 빈도를 계산합니다.
상관관계 시각화: 빈도수에 기반하여 단어 쌍의 상관관계를 시각화합니다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import combinations

# 1. 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 2. 재판당사자 열에서 단어 간의 앞뒤 상관관계 분석
def analyze_parties_relationship(data):
    # 재판당사자 열에서 단어 쌍 생성
    word_pairs = []
    for parties in data['재판당사자'].dropna():
        words = parties.split()  # 공백을 기준으로 단어 분리
        # 단어 간의 조합 생성 (앞뒤 관계)
        word_pairs.extend(combinations(words, 2))

    # 단어 쌍의 빈도 계산
    pair_counts = Counter(word_pairs)
    
    # 상위 10개의 단어 쌍 추출
    most_common_pairs = pair_counts.most_common(10)
    pairs, counts = zip(*most_common_pairs)

    # 단어 쌍 시각화
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=[' '.join(pair) for pair in pairs])
    plt.title('Top 10 Word Pairs in Parties')
    plt.xlabel('Frequency')
    plt.ylabel('Word Pairs')
    plt.show()

analyze_parties_relationship(data)


법원명에 따른 유사성 혹은 차별성 분석
 -법원별 사건명에서 주제 도출 판례내용_이유에서 유의미한 값 도출
 -도출된 주제에서 유사성 혹은 차별성 분석 

법원별 사건명 추출: 법원별로 사건명을 그룹화하여 판례내용_이유와 함께 사용합니다.
주제 도출: LDA(잠재 디리클레 할당) 모델을 사용하여 법원별로 주제를 도출합니다.
유사성 및 차별성 분석: 도출된 주제 간의 유사성 또는 차별성을 분석합니다. 이를 위해 코사인 유사도(cosine similarity)를 사용할 수 있습니다.

1. 데이터 준비 및 주제 도출

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 2. 법원별 사건명 및 판례내용_이유 추출
court_cases = data.groupby('법원명')['사건명', '판례내용_이유'].apply(lambda x: ' '.join(x['판례내용_이유'])).reset_index()
court_cases.columns = ['법원명', '판례내용_이유']

# 3. LDA 모델을 통해 주제 도출
def extract_topics(data, n_topics=5, n_top_words=10):
    vectorizer = CountVectorizer(stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(data['판례내용_이유'])

    lda_model = LDA(n_components=n_topics, random_state=42)
    lda_model.fit(doc_term_matrix)

    # 각 주제에 대한 단어 추출
    topics = []
    for idx, topic in enumerate(lda_model.components_):
        topic_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-n_top_words:]]
        topics.append(f'Topic {idx + 1}: ' + ', '.join(topic_words))
    
    return topics

topics = extract_topics(court_cases)
for topic in topics:
    print(topic)


2. 유사성 또는 차별성 분석

In [ ]:
# 4. 주제 간의 유사성 분석
def analyze_similarity(data, topics):
    topic_vectors = []
    vectorizer = CountVectorizer()
    for topic in topics:
        topic_words = topic.replace('Topic', '').split(': ')[1].split(', ')
        topic_vector = vectorizer.fit_transform([' '.join(topic_words)]).toarray()
        topic_vectors.append(topic_vector.flatten())

    # 코사인 유사도 계산
    similarity_matrix = cosine_similarity(topic_vectors)

    # 유사성 시각화
    plt.figure(figsize=(8, 6))
    sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm', xticklabels=[f'Topic {i + 1}' for i in range(len(topics))],
                yticklabels=[f'Topic {i + 1}' for i in range(len(topics))])
    plt.title('Topic Similarity Matrix')
    plt.show()

analyze_similarity(court_cases, topics)


 재판당사자별 빈도수 분석
- 사건별 재판당사자 수에 따른 특징 분석
- 재판당사자와 사건내용에 따른 상관관계 도출
- 네트워크 분석을 위한 인사이트 도출

1. 재판당사자별 빈도수 분석

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 재판당사자 열에서 단어별 빈도수 계산
def analyze_parties_frequency(data):
    # 재판당사자 열에서 모든 단어 추출
    all_parties = ' '.join(data['재판당사자'].dropna()).split()
    
    # 빈도수 계산
    party_counts = Counter(all_parties)
    
    # 상위 10개의 재판당사자 추출
    most_common_parties = party_counts.most_common(10)
    parties, counts = zip(*most_common_parties)

    # 빈도수 시각화
    plt.figure(figsize=(10, 6))
    plt.bar(parties, counts)
    plt.title('Top 10 Most Frequent Parties')
    plt.xlabel('Parties')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.show()

analyze_parties_frequency(data)


2. 사건별 재판당사자 수에 따른 특징 분석

In [ ]:
# 사건별 재판당사자 수 분석
def analyze_case_party_count(data):
    # 각 사건별 재판당사자 수 계산
    data['party_count'] = data['재판당사자'].apply(lambda x: len(str(x).split()))
    
    # 사건별 특징 분석
    case_party_counts = data.groupby('사건명')['party_count'].mean().reset_index()
    
    # 상위 10개의 사건 특징 분석
    top_cases = case_party_counts.nlargest(10, 'party_count')
    
    plt.figure(figsize=(10, 6))
    plt.barh(top_cases['사건명'], top_cases['party_count'])
    plt.title('Top 10 Cases by Average Number of Parties')
    plt.xlabel('Average Number of Parties')
    plt.ylabel('Cases')
    plt.show()

analyze_case_party_count(data)


3. 재판당사자와 사건내용에 따른 상관관계 도출

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

# 재판당사자와 사건내용 상관관계 분석
def analyze_parties_content_correlation(data):
    # 사건내용과 재판당사자 벡터화
    vectorizer = CountVectorizer()
    X_parties = vectorizer.fit_transform(data['재판당사자'].dropna())
    X_content = vectorizer.fit_transform(data['판례내용'].dropna())

    # Jaccard 유사도 계산
    correlation_matrix = []
    for i in range(X_parties.shape[0]):
        row_correlation = []
        for j in range(X_content.shape[0]):
            jaccard = jaccard_score(X_parties[i].toarray(), X_content[j].toarray())
            row_correlation.append(jaccard)
        correlation_matrix.append(row_correlation)

    plt.figure(figsize=(10, 6))
    sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f")
    plt.title('Correlation between Parties and Case Contents')
    plt.xlabel('Case Contents')
    plt.ylabel('Parties')
    plt.show()

analyze_parties_content_correlation(data)


4. 네트워크 분석을 위한 인사이트 도출

In [ ]:
import networkx as nx

# 네트워크 분석을 위한 인사이트 도출
def network_analysis(data):
    G = nx.Graph()
    
    # 재판당사자와 사건명 간의 관계 추가
    for idx, row in data.iterrows():
        parties = str(row['재판당사자']).split()
        case_name = row['사건명']
        
        # 재판당사자와 사건명 간의 엣지 생성
        for party in parties:
            G.add_edge(party, case_name)

    # 네트워크 시각화
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_size=100, node_color='blue', alpha=0.5)
    nx.draw_networkx_edges(G, pos, alpha=0.2)
    plt.title('Network of Parties and Cases')
    plt.show()

network_analysis(data)


판사, 변호사와 판례본문의 항목간의 상관관계 분석
-항목명 : 법원명, 사건종류명, 판결유형, 판시사항, 판결요지, 참조조문, 참조판례

1. 데이터 준비 및 상관관계 분석

2. 분석 설명
데이터 준비:

필요한 열을 추출하고 결측치를 제거합니다.
각 항목의 길이를 계산하여, 길이 간의 상관관계를 분석합니다. 이는 항목의 내용이 얼마나 길게 서술되는지를 확인하는 데 유용합니다.
상관관계 분석:

corr() 함수를 사용하여 각 열 간의 상관관계를 계산합니다.
시각화:

seaborn 라이브러리를 사용하여 상관관계 히트맵을 생성하고, 이를 통해 항목 간의 관계를 직관적으로 이해할 수 있도록 합니다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 필요한 항목 추출
required_columns = ['법원명', '사건종류명', '판결유형', '판시사항', '판결요지', '참조조문', '참조판례']
data_subset = data[required_columns]

# 결측치 제거
data_subset.dropna(inplace=True)

# 각 항목의 길이를 계산하여 상관관계 분석
data_subset['법원명_len'] = data_subset['법원명'].str.len()
data_subset['사건종류명_len'] = data_subset['사건종류명'].str.len()
data_subset['판결유형_len'] = data_subset['판결유형'].str.len()
data_subset['판시사항_len'] = data_subset['판시사항'].str.len()
data_subset['판결요지_len'] = data_subset['판결요지'].str.len()
data_subset['참조조문_len'] = data_subset['참조조문'].str.len()
data_subset['참조판례_len'] = data_subset['참조판례'].str.len()

# 상관관계 계산
correlation_matrix = data_subset.corr()

# 상관관계 시각화
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation between Judicial Attributes and Case Text')
plt.show()


'근로'에 대한 판례내용에서 등장하는 중심 단어와 그 주변 단어들의 네트워크 분석을 수행하려면 다음과 같은 단계로 진행할 수 있습니다:

데이터 필터링: '근로'와 관련된 판례 내용을 필터링합니다.
토큰화: 필터링된 판례 내용을 토큰화하여 단어를 추출합니다.
네트워크 구축: 중심 단어와 주변 단어 간의 관계를 기반으로 네트워크를 구축합니다.
네트워크 시각화: 네트워크를 시각화하여 중심 단어와 주변 단어 간의 관계를 확인합니다.

1. 데이터 필터링 및 토큰화

In [ ]:
import pandas as pd
import re
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# '근로' 관련 판례 내용 필터링
labor_cases = data[data['판례내용'].str.contains('근로', na=False)]

# 판례 내용에서 텍스트 추출 및 전처리
def preprocess_text(text):
    # 소문자 변환 및 특수문자 제거
    text = text.lower()
    text = re.sub(r'[^가-힣\s]', '', text)  # 한국어 및 공백만 남김
    return text

# 모든 판례 내용을 전처리
labor_cases['processed_text'] = labor_cases['판례내용'].apply(preprocess_text)

# 토큰화
all_words = ' '.join(labor_cases['processed_text']).split()


2. 중심 단어와 주변 단어 간의 네트워크 구축

In [ ]:
# 중심 단어 설정
central_word = '근로'

# 주변 단어 추출
def get_surrounding_words(words, central_word, window_size=2):
    central_indices = [i for i, word in enumerate(words) if word == central_word]
    surrounding_words = []

    for index in central_indices:
        start = max(0, index - window_size)
        end = min(len(words), index + window_size + 1)
        surrounding_words.extend(words[start:index] + words[index + 1:end])
    
    return surrounding_words

# 주변 단어 추출
surrounding_words = get_surrounding_words(all_words, central_word)

# 주변 단어 빈도수 계산
surrounding_counts = Counter(surrounding_words)

# 네트워크 구축
G = nx.Graph()

# 중심 단어와 주변 단어 간의 관계 추가
for word, count in surrounding_counts.items():
    G.add_edge(central_word, word, weight=count)

# 노드 크기 조정
node_sizes = [G.degree(n) * 100 for n in G.nodes()]


3. 네트워크 시각화

In [ ]:
# 네트워크 시각화
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='skyblue', alpha=0.7)
nx.draw_networkx_edges(G, pos, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=12)
plt.title('Network Analysis of Central and Surrounding Words Related to "근로"')
plt.axis('off')
plt.show()



판례 내용에서 산업군이나 특징적인 주제를 도출하기 위해 특정 주제어들을 추출하고 분석하는 방법은 다음과 같은 단계로 진행할 수 있습니다:

데이터 준비: 판례 내용을 로드하고 전처리합니다.
주제어 추출: 텍스트에서 중요한 주제어를 추출합니다.
주제어 분석: 주제어들의 빈도를 확인하고 분석합니다.
주제어 클러스터링: 주제어를 클러스터링하여 산업군 또는 특징적인 주제를 도출합니다.

1. 데이터 준비

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

# 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 판례 내용 추출
case_contents = data['판례내용'].dropna().tolist()

# 판례 내용 전처리
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^가-힣\s]', '', text)  # 한국어 및 공백만 남김
    return text

# 전처리된 판례 내용
processed_contents = [preprocess_text(content) for content in case_contents]


2. 주제어 추출

In [ ]:
# 주제어 추출을 위한 CountVectorizer 설정
vectorizer = CountVectorizer(max_features=100, stop_words='english')
X = vectorizer.fit_transform(processed_contents)

# 주제어 추출
feature_names = vectorizer.get_feature_names_out()
word_counts = X.toarray().sum(axis=0)

# 주제어와 그 빈도수
topic_words = pd.DataFrame({'주제어': feature_names, '빈도수': word_counts})
topic_words = topic_words[topic_words['빈도수'] > 0].sort_values(by='빈도수', ascending=False)

print(topic_words)


3. 주제어 분석
위의 코드에서 topic_words 데이터프레임은 주제어와 그 빈도수를 포함하고 있으며, 주제어의 빈도수를 출력합니다. 이 데이터프레임을 통해 가장 빈도가 높은 주제어를 확인할 수 있습니다.

4. 주제어 클러스터링
주제어 클러스터링을 위해 KMeans 알고리즘을 사용할 수 있습니다.

In [ ]:
from sklearn.cluster import KMeans

# KMeans 클러스터링
num_clusters = 5  # 원하는 클러스터 수
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# 클러스터 레이블
labels = kmeans.labels_

# 클러스터링 결과 추가
topic_words['클러스터'] = labels

# 각 클러스터의 주제어 출력
for cluster in range(num_clusters):
    print(f"\n클러스터 {cluster}:")
    print(topic_words[topic_words['클러스터'] == cluster]['주제어'].tolist())


KMeans 클러스터링을 사용하여 판례 내용에서 도출된 주제어를 분석할 때, 여러 가지 인사이트를 얻을 수 있습니다. 아래는 KMeans 클러스터링을 통해 도출할 수 있는 주요 인사이트입니다:

1. 산업군 및 주제 파악
산업군 식별: 각 클러스터에 포함된 주제어를 분석하여 해당 클러스터가 어떤 산업군에 속하는지 파악할 수 있습니다. 예를 들어, "근로", "임금", "계약"과 같은 단어가 클러스터에 포함되면, 이는 노동법 관련 주제를 나타낼 수 있습니다.
주제 구분: 판례가 다루는 다양한 주제를 명확히 구분할 수 있습니다. 예를 들어, 특정 클러스터가 법률적 분쟁에 관한 것이라면, 다른 클러스터는 규제나 정책과 관련된 내용을 포함할 수 있습니다.
2. 특징적인 패턴 분석
빈도수 및 경향 분석: 각 클러스터 내 주제어의 빈도를 확인함으로써 어떤 주제가 더 자주 다뤄지고 있는지를 알 수 있습니다. 이 정보는 법적 쟁점이나 산업 내 주요 이슈를 파악하는 데 유용합니다.
변화 추세 탐지: 시간에 따른 주제어의 변화를 추적하여 법률이나 산업의 변화, 사회적 트렌드를 분석할 수 있습니다.
3. 정책 및 전략적 인사이트
법률적 대응 방안: 특정 주제가 클러스터링에서 두드러지게 나타나면, 해당 주제에 대한 법률적 대응 방안이나 정책 개선이 필요할 수 있습니다.
리스크 관리: 특정 산업군에서 자주 발생하는 법적 쟁점이나 분쟁을 분석하여, 기업이나 법률 전문가가 리스크를 사전에 관리할 수 있는 전략을 마련할 수 있습니다.
4. 법률 연구 및 학술적 기초
연구 방향 제시: 각 클러스터의 주제어 분석을 통해 향후 연구가 필요한 법률 분야를 제안할 수 있습니다. 특히, 덜 다뤄진 주제나 이슈를 파악하여 심화 연구를 진행할 수 있습니다.
법률 교육 커리큘럼 개발: 교육 기관에서는 클러스터링 결과를 바탕으로 법률 교육 커리큘럼을 개발하여, 학생들이 필요로 하는 주제를 더 잘 다룰 수 있도록 할 수 있습니다.
5. 네트워크 분석 및 관계 도출
관계 탐색: 클러스터 내 주제어 간의 관계를 분석하여, 특정 주제들이 서로 어떻게 연관되는지를 이해할 수 있습니다. 이를 통해 주제어 간의 네트워크를 시각화하여 더 깊은 인사이트를 제공할 수 있습니다.
변호사 및 판사 관계: 클러스터링 결과를 통해 특정 산업군에서 자주 나타나는 변호사나 판사 간의 관계를 분석하여, 특정 분야에서의 영향력 있는 인물이나 그룹을 파악할 수 있습니다.

판례 내용에서 연관된 다양한 주제를 클러스터링하기 위해 KMeans 알고리즘을 활용하는 프로세스는 다음과 같습니다. 이 프로세스는 데이터를 준비하고, 텍스트를 전처리한 후, 주제어를 추출하고 클러스터링을 수행하는 단계로 구성됩니다.

아래의 단계별 코드를 통해 연관된 다양한 주제의 클러스터링을 도출할 수 있습니다.

1. 데이터 로드 및 전처리

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

# 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 판례 내용 추출
case_contents = data['판례내용'].dropna().tolist()

# 판례 내용 전처리
def preprocess_text(text):
    text = text.lower()  # 소문자 변환
    text = re.sub(r'[^가-힣\s]', '', text)  # 한국어 및 공백만 남김
    return text

# 전처리된 판례 내용
processed_contents = [preprocess_text(content) for content in case_contents]


2. 주제어 추출 및 벡터화

In [ ]:
# 주제어 추출을 위한 CountVectorizer 설정
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(processed_contents)

# 주제어 추출
feature_names = vectorizer.get_feature_names_out()
word_counts = X.toarray().sum(axis=0)

# 주제어와 그 빈도수
topic_words = pd.DataFrame({'주제어': feature_names, '빈도수': word_counts})
topic_words = topic_words[topic_words['빈도수'] > 0].sort_values(by='빈도수', ascending=False)

print("주제어 빈도수:")
print(topic_words)


3. KMeans 클러스터링 수행

In [ ]:
from sklearn.cluster import KMeans

# KMeans 클러스터링
num_clusters = 5  # 원하는 클러스터 수
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# 클러스터 레이블
labels = kmeans.labels_

# 클러스터링 결과 추가
topic_words['클러스터'] = labels


4. 클러스터 결과 분석 및 출력

In [ ]:
# 각 클러스터의 주제어 출력
for cluster in range(num_clusters):
    print(f"\n클러스터 {cluster}:")
    print(topic_words[topic_words['클러스터'] == cluster]['주제어'].tolist())


도출된 주제의 핵심 키워드와 연관된 많이 사용하는 용어들을 출력하고, 이를 네트워크로 시각화하며, 또한 워드 클라우드로 표현하기 위한 단계는 다음과 같습니다.

1. 데이터 준비 및 전처리
이전 단계에서 판례 내용을 전처리하고 KMeans 클러스터링을 수행한 후, 각 클러스터의 키워드와 관련 용어를 추출해야 합니다. 아래 코드는 이를 수행하는 방법을 보여줍니다.

2. 핵심 키워드 추출
각 클러스터에서 핵심 키워드를 출력하고, 이를 기반으로 연관된 용어를 추출합니다

In [ ]:
# 각 클러스터의 주제어 출력 및 연관 단어 추출
cluster_keywords = {}

for cluster in range(num_clusters):
    keywords = topic_words[topic_words['클러스터'] == cluster]['주제어'].tolist()
    cluster_keywords[cluster] = keywords
    print(f"\n클러스터 {cluster}의 핵심 키워드: {keywords}")

    # 연관 단어 추출
    for keyword in keywords:
        # 각 키워드와 관련된 용어 추출 (여기서는 빈도수 기준으로 정렬된 단어를 사용)
        related_words = topic_words[topic_words['주제어'].str.contains(keyword)]['주제어'].tolist()
        print(f"'{keyword}'와 연관된 용어: {related_words}")


3. 네트워크 시각화
핵심 키워드와 연관된 용어를 네트워크로 시각화하기 위해 networkx 및 matplotlib 라이브러리를 사용할 수 있습니다.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# 네트워크 그래프 생성
G = nx.Graph()

# 클러스터 내 키워드 및 연관 단어 간의 관계 추가
for cluster, keywords in cluster_keywords.items():
    for keyword in keywords:
        G.add_node(keyword)  # 키워드 노드 추가
        for related in topic_words[topic_words['주제어'].str.contains(keyword)]['주제어']:
            G.add_node(related)  # 연관 단어 노드 추가
            G.add_edge(keyword, related)  # 간선 추가

# 그래프 시각화
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightblue', font_size=10, font_color='black', font_weight='bold', edge_color='gray')
plt.title('주요 키워드와 연관된 용어 네트워크', fontsize=16)
plt.show()


4. 워드 클라우드 시각화
워드 클라우드를 생성하기 위해 WordCloud 라이브러리를 사용할 수 있습니다.

In [ ]:
from wordcloud import WordCloud

# 전체 텍스트로 워드 클라우드 생성
all_keywords = ' '.join(topic_words['주제어'].tolist())
wordcloud = WordCloud(font_path='path_to_korean_font.ttf', width=800, height=400, background_color='white').generate(all_keywords)

# 워드 클라우드 시각화
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('주요 키워드 워드 클라우드', fontsize=16)
plt.show()


사건번호와 선고일자에 따른 참조판례 및 참조조문의 변화를 시계열 분석하고, 이를 시각화하는 방법에 대한 설명을 아래에 제공합니다. 이 과정에서는 pandas와 matplotlib를 사용하여 데이터를 처리하고 시각화합니다.

1. 데이터 준비 및 전처리
우선, 필요한 데이터를 로드하고 전처리하는 과정을 거칩니다. 여기서는 사건번호, 선고일자, 참조판례, 참조조문이 포함된 데이터프레임을 가정합니다.

In [ ]:
import pandas as pd
import re

# 데이터 로드
data = pd.read_csv('service.csv', encoding='utf-8')

# 필요한 열 추출 및 결측치 제거
data = data[['사건번호', '선고일자', '참조판례', '참조조문']].dropna()

# 선고일자 데이터 형식 변환
data['선고일자'] = pd.to_datetime(data['선고일자'], errors='coerce')

# 데이터 확인
print(data.head())


2. 참조판례 및 참조조문 빈도수 집계
사건번호와 선고일자에 따른 참조판례 및 참조조문의 빈도수를 집계합니다.

In [ ]:
# 선고일자에 따른 참조판례 및 참조조문 빈도수 집계
reference_cases = data.groupby(['선고일자', '참조판례']).size().reset_index(name='빈도수')
reference_statutes = data.groupby(['선고일자', '참조조문']).size().reset_index(name='빈도수')

# 결과 확인
print(reference_cases.head())
print(reference_statutes.head())


3. 시계열 분석
시계열 데이터를 분석하기 위해 월별 혹은 연도별로 집계하여 시계열 변화를 확인합니다

In [ ]:
# 월별 집계
reference_cases['월'] = reference_cases['선고일자'].dt.to_period('M')
reference_statutes['월'] = reference_statutes['선고일자'].dt.to_period('M')

# 월별 참조판례 빈도수 집계
monthly_cases = reference_cases.groupby(['월', '참조판례']).sum().reset_index()
monthly_statutes = reference_statutes.groupby(['월', '참조조문']).sum().reset_index()

# 결과 확인
print(monthly_cases.head())
print(monthly_statutes.head())


4. 시각화
이제 집계된 결과를 바탕으로 시계열 데이터를 시각화합니다.

In [ ]:
import matplotlib.pyplot as plt

# 참조판례 시각화
plt.figure(figsize=(14, 7))
for case in monthly_cases['참조판례'].unique():
    subset = monthly_cases[monthly_cases['참조판례'] == case]
    plt.plot(subset['월'].dt.to_timestamp(), subset['빈도수'], marker='o', label=case)

plt.title('사건번호 및 선고일자에 따른 참조판례 변화 시계열')
plt.xlabel('선고일자')
plt.ylabel('빈도수')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

# 참조조문 시각화
plt.figure(figsize=(14, 7))
for statute in monthly_statutes['참조조문'].unique():
    subset = monthly_statutes[monthly_statutes['참조조문'] == statute]
    plt.plot(subset['월'].dt.to_timestamp(), subset['빈도수'], marker='o', label=statute)

plt.title('사건번호 및 선고일자에 따른 참조조문 변화 시계열')
plt.xlabel('선고일자')
plt.ylabel('빈도수')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


5. 시간 혹은 날짜에 대한 정보 도출
각 참조판례와 참조조문에서 시간이나 날짜에 대한 정보를 도출하기 위해 날짜 관련 메타데이터를 추출할 수 있습니다.

python

In [ ]:
# 시간 및 날짜 정보 추출 함수
def extract_date_info(ref_column):
    date_info = []
    for ref in ref_column:
        # 참조판례 혹은 참조조문에서 날짜 정보 추출 (여기서는 예시로 'YYYY-MM-DD' 형식을 찾음)
        dates = re.findall(r'\d{4}-\d{2}-\d{2}', ref)
        date_info.extend(dates)
    return date_info

# 참조판례 및 참조조문에서 날짜 정보 추출
case_dates = extract_date_info(data['참조판례'])
statute_dates = extract_date_info(data['참조조문'])

# 결과 확인
print("참조판례에서 추출된 날짜 정보:", case_dates)
print("참조조문에서 추출된 날짜 정보:", statute_dates)


#### 전처리 순서